In [1]:
!pip install kmeans-pytorch
!pip install modin[ray]
!pip install recbole

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 MB 10.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 2.2.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64

In [2]:
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.general_recommender import BPR
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger

In [3]:
config = Config(model="BPR", dataset="ml-100k")
config


General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = /usr/local/lib/python3.10/dist-packages/recbole/config/../dataset_example/ml-100k
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'RO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_de

In [6]:
init_seed(config["seed"], config["reproducibility"])

In [7]:
init_logger(config)

In [9]:
logger = getLogger()
logger

<RootLogger root (WARNING)>

In [11]:
logger.info(config)

In [12]:
dataset = create_dataset(config)
logger.info(dataset)

/usr/local/lib/python3.10/dist-packages/recbole/data/dataset/dataset.py:648: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  feat[field].fillna(value=0, inplace=True)
/usr/local/lib/python3.10/dist-packages/recbole/data/dataset/dataset.py:650: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

In [13]:
train_data, valid_data, test_data = data_preparation(config, dataset)

In [16]:
train_data, len(train_data)

(<recbole.data.dataloader.general_dataloader.TrainDataLoader at 0x794c90606620>,
 40)

In [21]:
for i in train_data:
  print(i)
  break

The batch_size of interaction: 2048
    user_id, torch.Size([2048]), cpu, torch.int64
    item_id, torch.Size([2048]), cpu, torch.int64
    rating, torch.Size([2048]), cpu, torch.float32
    timestamp, torch.Size([2048]), cpu, torch.float32
    neg_item_id, torch.Size([2048]), cpu, torch.int64




In [22]:
train_data.dataset

ml-100k
The number of users: 944
Average actions of users: 85.69247083775186
The number of items: 1683
Average actions of items: 48.826586102719034
The number of inters: 80808
The sparsity of the dataset: 94.91374361763195%
Remain Fields: ['user_id', 'item_id', 'rating', 'timestamp', 'neg_item_id']

In [23]:
model = BPR(config, train_data.dataset).to(config["device"])
logger.info(model)

In [24]:
trainer = Trainer(config, model)

In [25]:
best_valid_score, best_valid_result = trainer.fit(train_data, valid_data)
best_valid_score

0.3728

In [26]:
best_valid_result

OrderedDict([('recall@10', 0.2073),
             ('mrr@10', 0.3728),
             ('ndcg@10', 0.2255),
             ('hit@10', 0.7423),
             ('precision@10', 0.1591)])

In [27]:
test_result = trainer.evaluate(test_data)

In [28]:
test_result

OrderedDict([('recall@10', 0.2444),
             ('mrr@10', 0.4864),
             ('ndcg@10', 0.29),
             ('hit@10', 0.7826),
             ('precision@10', 0.1955)])